In [53]:
# Import stuff <3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Find our letters
group_nr = 2
letters = "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]

base =  "https://en.wikinews.org/"

# List with URLs depend on letters
url_list = []
for l in letters:
    url_list.append('https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from={}'.format(l))
    
# Loop through url list and print all titles
for url,l in zip(url_list,letters):
    
    # Needed for doing multiple passes
    all_found = False
    page_url = url
    all_titles = []
    
    # While there are still some articles not found
    while not all_found:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        container = soup.find(id = "mw-pages")
        
        data = container.find_all('li')
        
        # Added so it only takes titles with the correct first letter
        titles = [m.text for m in data if m.text[0].upper()==l]
        
        # Append the titles to the running list
        all_titles += titles
        
        # If the last element on the page is not part of our letter we go to the next
        if data[-1].text[0] != l:
            break
            
        # Otherwise we find the "next page" button and follows it
        buttons = soup.findAll('a', href=True)
        for b in buttons:
            try:
                if b.contents[0] == "next page":
                    page_url = base + b["href"]
            except:
                pass
            


208